In [10]:
pip install git+https://github.com/HOUPU1993/tidyzoning.git

  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-tq5dgx9h
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit 5bb98f72521434fad27f08a0ca5ea7965896c73d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-tq5dgx9h'


In [1]:
import tidyzoning

In [2]:
tidyzoning.custom_formula(4)

AttributeError: module 'tidyzoning' has no attribute 'custom_formula'

In [ ]:
hello_world()

NameError: name 'hello_world' is not defined

In [5]:
from tidyzoning import custom_formula

ImportError: cannot import name 'custom_formula' from 'tidyzoning' (c:\Users\Admin\anaconda3\envs\houpu_py\lib\site-packages\tidyzoning\__init__.py)